In [5]:
#修飾器通常會將函式換成不同函式

def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running terget')
    

target()
#target已經參考到inner
print(target)


running inner()
<function deco.<locals>.inner at 0x7fe8882aa048>


In [27]:
import registration


#registration.registry
#registration.f1()

registration.register(registration.f1)

running registry(<function f1 at 0x7fe893920bf8>)


<function registration.f1()>

In [4]:
# 7-4 讀取一個區域與一個全域變數的函式
def f1(a):
    print(a)
    print(b)
    
b =6 
f1('w')

#7-5 變數b是區域變數,因為他在函式的內文中被賦值

b=6
def f2(a):
    print(a)
    print(b)
    b= 9

w
6


In [8]:
#比較Bytecode
#7-6 拆解7-4的1f1函式
def f1(a):
    print(a)
    print(b)

from dis import dis
dis(f1)

def f2(a):
    print(a)
    print(b)
    b= 9
print('----------------------------------------------------------')
#7-7 拆解7-5的f2函式
dis(f2)


  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
----------------------------------------------------------
 11           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

 12           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

 13          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [17]:
from average_oo import Average

avg = Average()

avg(10)


import os

print(os.environ['avg'])

%reset


KeyError: 'avg'

In [24]:
# %load average.py
#
#average_oo.py:更高階的移動平均值的計算函式
#%reset

def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total =sum(series)
        return total/len(series)

    return averager



avg = make_averager()


avg(10)

#Returns a tuple containing 
#the names of the local variables (starting with the argument names)
avg.__code__.co_varnames


#co_freevars:Returns a tuple containing the names of free variables.

avg.__code__.co_freevars

#7-12 延續7-10

avg.__code__.co_freevars

avg.__closure__
avg.__closure__[0].cell_contents
                                                                                                                                                                        

[10]

In [25]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count,total
        count += 1
        total += new_value
        return total/count

    return averager



In [1]:
import time

to = time.perf_counter()

elapsed = time.perf_counter() -to

elapsed


0.00011959699986618944

In [3]:
#7-16 使用clock修飾器

#clockdeco_demo.py
import time

from clockdeco_demo import clock

@clock
def snooze(seconds):
    time.sleep(seconds)
@clock 
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*'*40,'Calling snooze(.123)')
snooze(.123)
print('*'*40,'Calling factorial(6)')
print('6!=',factorial(6))



**************************************** Calling snooze(.123)
[0.12314s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000s] factorial(1) -> 1
[0.00002s] factorial(2) -> 2
[0.00002s] factorial(3) -> 6
[0.00003s] factorial(4) -> 24
[0.00004s] factorial(5) -> 120
[0.00005s] factorial(6) -> 720
6!= 720


In [4]:
'''
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

為

def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

factorial = clock(factorial) 
'''






SyntaxError: invalid syntax (<ipython-input-4-6588acdea05e>, line 1)

In [8]:
import clockdemo

clockdemo.factorial.__name__


'clocked'

In [9]:
#7-17 改善後的clock修飾器
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args,**kwargs):
        t0 = time.time()
        result = func(*args,**kwargs)
        elapsed = time.time() -t0
        name = func.__neme__
        arg_1st = []
        if args:
            arg_1st.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_1st.append(', '.join(pairs))
        arg_str = ', '.join(arg_1st)
        print('[%0.8fs] %s(%s) -> %r'%(elapsed,name,arg_str,result))
        return result
    return clocked


In [12]:
import time

def add(a,b):
    print(a+b)
    
def timer(func,*args):
    start = time.time()
    func(*args)
    time.sleep(2)
    long = time.time()- start
    print('共耗時%f秒' %long)
    
timer(add,1,2)


    
    

3
共耗時2.000322秒


In [2]:
from functools import wraps

def my_decorator(f):
    @wraps(f)
    def wrapper(*args,**kwds):
        print('Calling decorated function')
        return f(*args,**kwds)
    return wrapper

@my_decorator
def example():
    """Docsting"""
    print('Called example function')
   

'''
def example


example = up


'''
example()

Calling decorated function
Called example function


In [3]:
#7-19 使用cache,更快速的作法

import functools

from clockdeco_demo import clock

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n;
    return (fibonacci(n-2)+fibonacci(n-1))

if __name__=='main':
    print(fibonacci(6))
    
    



In [12]:
import html 

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

htmlize({1,2,3})

htmlize(abs)

htmlize('Heimlich & Co.\n- a game')

htmlize(42)


'<pre>42</pre>'

In [18]:
#functools.singledispatch
#Transform a function into a single-dispatch generic function

from functools import singledispatch

@singledispatch
def fun(arg,verbose=False):
    if verbose:
        print("Let me just say",end=" ")
    print(arg)
    

@fun.register
def AA(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)

@fun.register(complex)
def _(arg, verbose=False):
    if verbose:
        print("Better than complicated.", end=" ")
    print(arg.real, arg.imag)







NameError: name 'eee' is not defined

In [ ]:
#7-21 singledispatch 建立一個自訂的htmlize.register,來將許多函式包成一個通用函式 

from functools import singledispatch

from collections import abc
